In [1]:
import pandas as pd
import numpy as np
from transformers import ( AutoModelForSeq2SeqLM, AutoTokenizer,MBartForConditionalGeneration,
                         pipeline)
import torch
import os

/home/phoenix/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-14 18:23:33.231105: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 18:23:33.231184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 18:23:33.262857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 18:23:33.328920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is opt

In [ ]:
def extract_summary(generated_text):
    prefix_to_remove = "Summary :"
    prefix_to_remove2 = "Summary:"
    generated_text=str(generated_text)
    if generated_text.startswith(prefix_to_remove):
        cleaned_text = generated_text[len(prefix_to_remove):]
    else:
        cleaned_text = generated_text
    if generated_text.startswith(prefix_to_remove2):
        cleaned_text2 = cleaned_text[len(prefix_to_remove2):]
    else:
        cleaned_text2 = cleaned_text
    return cleaned_text2

In [ ]:
def process_news_prompt(prompt, max_tensor_size=1024):
    inputs = tokenizer(prompt, return_tensors="tf")

    # Get the size of the input tensor
    tensor_size = inputs["input_ids"].shape[1]
    print(tensor_size)
    # Check if the tensor size is greater than 1024
    if tensor_size > max_tensor_size:
        # Shorten the prompt to fit within 1024 tokens
        num_tokens_to_remove = tensor_size - max_tensor_size
        prompt = tokenizer.decode(inputs["input_ids"][0, :-num_tokens_to_remove])
    prefix_to_remove = "en_XX "
    if prompt.startswith(prefix_to_remove):
        cleaned_text = prompt[len(prefix_to_remove):]
    else:
        cleaned_text = prompt
    # Return the reduced prompt as text
    return cleaned_text

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,MBartForConditionalGeneration

In [3]:
model_ckpt = "finetunedmBartLarge67kWith50news147Scripts"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = MBartForConditionalGeneration.from_pretrained(model_ckpt)

In [6]:
prompt = """Text :اردو زبان کو اُردو تختی اور جموں کشمیر کی ادبی زبان قرار دیا جاتا ہے۔ یہ زبان مشرقی پاکستان میں بولی جاتی ہے اور اس کے علاوہ ہندوستان کے کئی حصوں میں بھی استعمال ہوتی ہے۔ اردو کا تعلق عربی اور فارسی زبانوں سے ہے، اور اس کا لکھائی ادبی عربی سے مشابہت رکھتا ہے۔ اس کی لکھائی نستعلیق، نستعلیق نگار، اور نستعلیقِ نگار نستعلیق کہلاتی ہیں۔

اردو زبان کی تاریخ بہت پرانی ہے اور اس کی شروعات قدیم ہندوستانی زبانوں سے ملتی ہے۔ اردو کی اختصاری شکل ہے 'اردو' جو 'اردو زبان کا مخفف ہے۔ اردو میں نثر اور نظم کا بہترین اظہار کیا جاتا ہے، اور اس زبان کے شاعر وادیئے، میرزا غالب، اور فیض احمد فیض جیسے معروف ادیب بڑے مقام حاصل کر چکے ہیں۔

اس دورانِ حاضر، ٹیکنالوجی کے فیصلے کرنے والے لوگ بھی اردو کے استعمال کے فوائد سمجھ رہے ہیں۔ اردو میں ٹیکسٹ سمارٹ فون، کمپیوٹر، اور انٹرنیٹ کی مختلف اپلیکیشنز کے لئے استعمال ہوتا ہے۔ اردو کے لئے بھی مخصوص نئے الفاظ اور ٹیکنالوجی کے مصطلحات تیار کیے جا رہے ہیں تاکہ زبان کی ترویج اور ترقی میں مدد فراہم کی جائے۔"""

In [4]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [7]:
# prompt=dataset_test['Question'][9]
prompt='Text :'+prompt
prompt

"Text :Text :اردو زبان کو اُردو تختی اور جموں کشمیر کی ادبی زبان قرار دیا جاتا ہے۔ یہ زبان مشرقی پاکستان میں بولی جاتی ہے اور اس کے علاوہ ہندوستان کے کئی حصوں میں بھی استعمال ہوتی ہے۔ اردو کا تعلق عربی اور فارسی زبانوں سے ہے، اور اس کا لکھائی ادبی عربی سے مشابہت رکھتا ہے۔ اس کی لکھائی نستعلیق، نستعلیق نگار، اور نستعلیقِ نگار نستعلیق کہلاتی ہیں۔\n\nاردو زبان کی تاریخ بہت پرانی ہے اور اس کی شروعات قدیم ہندوستانی زبانوں سے ملتی ہے۔ اردو کی اختصاری شکل ہے 'اردو' جو 'اردو زبان کا مخفف ہے۔ اردو میں نثر اور نظم کا بہترین اظہار کیا جاتا ہے، اور اس زبان کے شاعر وادیئے، میرزا غالب، اور فیض احمد فیض جیسے معروف ادیب بڑے مقام حاصل کر چکے ہیں۔\n\nاس دورانِ حاضر، ٹیکنالوجی کے فیصلے کرنے والے لوگ بھی اردو کے استعمال کے فوائد سمجھ رہے ہیں۔ اردو میں ٹیکسٹ سمارٹ فون، کمپیوٹر، اور انٹرنیٹ کی مختلف اپلیکیشنز کے لئے استعمال ہوتا ہے۔ اردو کے لئے بھی مخصوص نئے الفاظ اور ٹیکنالوجی کے مصطلحات تیار کیے جا رہے ہیں تاکہ زبان کی ترویج اور ترقی میں مدد فراہم کی جائے۔"

In [8]:
size=tokenizer(prompt, return_tensors="tf")["input_ids"].shape[1]
size

2023-12-14 18:25:14.476032: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 18:25:14.484073: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 18:25:14.484259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

250

In [9]:
from transformers import pipeline

In [ ]:
# from flask import Flask, render_template, request

# app = Flask(__name__)

# @app.route('/')
# def index():
#     return render_template('index.html')

# @app.route('/process', methods=['POST'])
# def process():
#     input_text = request.form['inputText']
#     # Call your model or processing logic here
#     output_text = f"Processed: {input_text}"
#     generator = pipeline('text2text-generation', model=model,device="cuda", tokenizer=tokenizer,do_sample=False)
#     result = generator(prompt, max_length=size+100)
#     return render_template('app/app/templates/index.html', input_text=input_text, output_text=result)

# if __name__ == '__main__':
#     app.run(debug=True)

In [12]:
generator = pipeline('text2text-generation', model=model,device="cuda:0", tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_length=size+100)

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacty of 11.76 GiB of which 28.50 MiB is free. Including non-PyTorch memory, this process has 11.46 GiB memory in use. Of the allocated memory 1.46 GiB is allocated by PyTorch, and 14.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
result